In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold # pip3 install --user iterative-stratification on SCC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix

import random
import math
from collections import Counter

In [2]:
df = pd.read_excel('../Data/growth_profiles_binary.xlsx',
                     index_col='strain')  # ,index_col='Mutant'

df_new = df.astype(float)
X = df_new.values
n, d = X.shape
"""## only for classify, 0 -->-1 """

X[X == 0] = -1

df.iloc[0:3,:]

,difcoMB,HMBcmpt,HMBpep,HMBaa,HMBlips,HMBoligo,HMBorg,HMBntrl,HMBamisug,HMBacdsug,HMB--
strain,,,,,,,,,,,
fang,1,0,1,1,1,0,0,0,0,0,0
kordia,1,0,0,0,1,0,0,0,0,0,0
citrea,1,1,1,1,1,1,1,1,1,1,0


In [3]:
df.std(axis=1)

strain
fang       0.504525
kordia     0.404520
citrea     0.301511
kuro       0.301511
HOT1A3     0.000000
             ...   
HOT5_C3    0.404520
luca8      0.522233
HOT5_E6    0.467099
DFL-12     0.522233
DSS-3      0.522233
Length: 65, dtype: float64

In [4]:
X=df.values
n, d = X.shape
print((n, d))

(65, 11)


In [5]:
X[X==0]=-1
print(X.dtype)##dtype('int64') 
X

int64


array([[ 1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1],
       [ 1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1],
       [ 1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1],
       [ 1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1],
       [ 1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1,  1, -1, -1,  1,  1, -1, -1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1],
       [ 1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 1, -1,  1,  1,  1, -1,  1, -1,  1, -1

## Random Forest classifier

In [6]:
df_z_all=pd.read_csv('../Results/MIP_classify_z_all_11.csv', dtype = 'Int64')
df_z_all.head()

,difcoMB,HMBcmpt,HMBpep,HMBaa,HMBlips,HMBoligo,HMBorg,HMBntrl,HMBamisug,HMBacdsug,HMB--
0,0,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,0,1,1
2,0,0,1,1,1,1,1,1,0,1,1
3,0,1,0,1,1,0,1,1,0,1,1
4,0,0,1,1,1,1,1,1,0,0,0


In [7]:
df_z_all.shape

(10, 11)

In [9]:
names_raw=df.columns.values
n, d = X.shape  

#z_last = df_z_all.iloc[-1,:].values
#y_last = X[:,z_last==0] 


myseed = 2020
cvFolds= 5

In [10]:
def sample_attrs(attrs, r, max=100, seed = 1234567890):
    i = 0
    random.seed(seed)
    while i < max:
        preds = random.sample(attrs, r)
        preds.sort()
        resps = list(set(attrs) - set(preds))
        yield (preds, resps)
        i += 1

In [11]:
def entropy(data):
    l = len(data)
    if l <= 1:
        return 0
    counts = Counter()
    for d in data:
        counts[d] += 1
    ent = 0
    probs = [float(c) / len(data) for c in counts.values()]
    for p in probs:
        ent -= p * math.log(p, 2.)
    return ent

x_idxs_byentropy = df.apply(entropy, 0).reset_index().sort_values(0, ascending=False)
x_idxs_byentropy = list(x_idxs_byentropy.index)

In [17]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, jaccard_score, cohen_kappa_score, matthews_corrcoef

stats_all=[]
acc_all=[]
b_acc_all=[]
f1_weight_all=[]
jaccard_all=[]
cohen_kappa_all=[]
mcc_all=[]
mcc_js=[]

attrs_pred_max = 300

attrs_idx = list(range(df_z_all.shape[1]))

for r in range(1, df_z_all.shape[1]):
    # for cross-validation
    if r==df_z_all.shape[1]-1:
        folds = StratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    else:
        folds = MultilabelStratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    
    # MASS-selected attributes
    z=df_z_all.iloc[df_z_all.shape[0]-r,:].values
    x_idxs_MASS = [i for i, _ in enumerate(z) if _ == 1]
    y_idxs_MASS = [i for i, _ in enumerate(z) if _ == 0]

    # max-entropy-selected attributes
    x_idxs_ent = x_idxs_byentropy[:r]
    x_idxs_ent.sort()
    y_idxs_ent = list(set(attrs_idx) - set(x_idxs_ent))
    
    # draw random attributes
    if r == 1:
        attrs_remaining = list(set(attrs_idx) - set(x_idxs_MASS))
        idxs_rand = [([x], list(set(attrs_idx) - set([x]))) for x in attrs_remaining]
    else:
        idxs_rand = list(sample_attrs(attrs_idx, r, attrs_pred_max, myseed))

    # first list item is the MASS-selected attr set
    x_idxs = [x_idxs_MASS] + [x_idxs_ent] + [xx[0] for xx in idxs_rand]
    y_idxs = [y_idxs_MASS] + [y_idxs_ent] + [xx[1] for xx in idxs_rand]

    for i in range(len(x_idxs)):
        x_idx = x_idxs[i]
        y_idx = y_idxs[i]
        x_idx_short = '|'.join([str(x) for x in x_idx])

        y_split = X[:,y_idx]
        name_i = names_raw[y_idx]
    
        for fold_, (trn_idx, test_idx) in enumerate(folds.split(X,y_split)):        
            print('r, i, fold_, and x_idx==', r, i, fold_, x_idx_short, end=';  ')  
            train_X, test_X = X[trn_idx][:,x_idx], X[test_idx][:,x_idx]
            train_y, test_y = X[trn_idx][:,y_idx], X[test_idx][:,y_idx]
        
            if fold_==0:#,name_i
                print('train_y.shape, train_y/test_y.mean:',train_y.shape, round(train_y.mean(),2), round(test_y.mean(),2))
        
            y_te_pre = []
            for j in range(train_y.shape[1]):
                rfc=RandomForestClassifier(n_estimators=10, random_state=myseed,n_jobs=5,class_weight="balanced")
                rfc.fit(train_X, train_y[:,j])
                y_te=rfc.predict(test_X)
                y_te_pre.append(y_te)
                if i==0 or r==df_z_all.shape[1]: # only record response-specific performances for the MASS predictor set, and the control
                    #print("j=",j, "; x_idx=",x_idx, "; y_idx=", y_idx, "; response=", names_raw[y_idx[j]])
                    mcc_js.append([r, i, x_idx_short, names_raw[y_idx[j]], fold_,
                                   matthews_corrcoef(test_y[:,j], y_te)])
            y_te_pre = np.array(y_te_pre).T
            tn_fp_fn_tp = confusion_matrix(test_y.ravel(), y_te_pre.ravel()).ravel()
            tn_fp_fn_tp = '|'.join( [str(x) for x in tn_fp_fn_tp] )
            
            stats_all.append([r, i, x_idx_short,fold_, tn_fp_fn_tp])
            acc_all.append([r, i, x_idx_short,fold_, accuracy_score(test_y.ravel(), y_te_pre.ravel())])
            b_acc_all.append([r, i, x_idx_short,fold_, balanced_accuracy_score(test_y.ravel(), y_te_pre.ravel())])
            f1_weight_all.append([r, i, x_idx_short,fold_, f1_score(test_y.ravel(), y_te_pre.ravel(), average='weighted')])
            jaccard_all.append([r, i, x_idx_short,fold_, jaccard_score(test_y.ravel(), y_te_pre.ravel(), average='weighted')])
            cohen_kappa_all.append([r, i, x_idx_short,fold_, cohen_kappa_score(test_y.ravel(), y_te_pre.ravel())])
            mcc_all.append([r, i, x_idx_short,fold_, matthews_corrcoef(test_y.ravel(), y_te_pre.ravel())])

r, i, fold_, and x_idx== 1 0 0 5;  train_y.shape, train_y/test_y.mean: (52, 10) 0.1 0.23
r, i, fold_, and x_idx== 1 0 1 5;  r, i, fold_, and x_idx== 1 0 2 5;  r, i, fold_, and x_idx== 1 0 3 5;  r, i, fold_, and x_idx== 1 0 4 5;  r, i, fold_, and x_idx== 1 1 0 5;  train_y.shape, train_y/test_y.mean: (52, 10) 0.1 0.23
r, i, fold_, and x_idx== 1 1 1 5;  r, i, fold_, and x_idx== 1 1 2 5;  r, i, fold_, and x_idx== 1 1 3 5;  r, i, fold_, and x_idx== 1 1 4 5;  r, i, fold_, and x_idx== 1 2 0 0;  train_y.shape, train_y/test_y.mean: (52, 10) -0.0 0.2
r, i, fold_, and x_idx== 1 2 1 0;  r, i, fold_, and x_idx== 1 2 2 0;  r, i, fold_, and x_idx== 1 2 3 0;  r, i, fold_, and x_idx== 1 2 4 0;  r, i, fold_, and x_idx== 1 3 0 1;  train_y.shape, train_y/test_y.mean: (52, 10) 0.06 0.26
r, i, fold_, and x_idx== 1 3 1 1;  r, i, fold_, and x_idx== 1 3 2 1;  r, i, fold_, and x_idx== 1 3 3 1;  r, i, fold_, and x_idx== 1 3 4 1;  r, i, fold_, and x_idx== 1 4 0 2;  train_y.shape, train_y/test_y.mean: (52, 10) 0.0

r, i, fold_, and x_idx== 2 22 1 8|10;  r, i, fold_, and x_idx== 2 22 2 8|10;  r, i, fold_, and x_idx== 2 22 3 8|10;  r, i, fold_, and x_idx== 2 22 4 8|10;  r, i, fold_, and x_idx== 2 23 0 4|7;  train_y.shape, train_y/test_y.mean: (52, 9) 0.09 0.28
r, i, fold_, and x_idx== 2 23 1 4|7;  r, i, fold_, and x_idx== 2 23 2 4|7;  r, i, fold_, and x_idx== 2 23 3 4|7;  r, i, fold_, and x_idx== 2 23 4 4|7;  r, i, fold_, and x_idx== 2 24 0 3|4;  train_y.shape, train_y/test_y.mean: (52, 9) 0.04 0.3
r, i, fold_, and x_idx== 2 24 1 3|4;  r, i, fold_, and x_idx== 2 24 2 3|4;  r, i, fold_, and x_idx== 2 24 3 3|4;  r, i, fold_, and x_idx== 2 24 4 3|4;  r, i, fold_, and x_idx== 2 25 0 7|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.15 0.33
r, i, fold_, and x_idx== 2 25 1 7|8;  r, i, fold_, and x_idx== 2 25 2 7|8;  r, i, fold_, and x_idx== 2 25 3 7|8;  r, i, fold_, and x_idx== 2 25 4 7|8;  r, i, fold_, and x_idx== 2 26 0 3|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.1 0.35
r, i, fold_, and x_idx

r, i, fold_, and x_idx== 2 56 1 4|10;  r, i, fold_, and x_idx== 2 56 2 4|10;  r, i, fold_, and x_idx== 2 56 3 4|10;  r, i, fold_, and x_idx== 2 56 4 4|10;  r, i, fold_, and x_idx== 2 57 0 1|10;  train_y.shape, train_y/test_y.mean: (52, 9) 0.14 0.32
r, i, fold_, and x_idx== 2 57 1 1|10;  r, i, fold_, and x_idx== 2 57 2 1|10;  r, i, fold_, and x_idx== 2 57 3 1|10;  r, i, fold_, and x_idx== 2 57 4 1|10;  r, i, fold_, and x_idx== 2 58 0 1|5;  train_y.shape, train_y/test_y.mean: (52, 9) 0.09 0.21
r, i, fold_, and x_idx== 2 58 1 1|5;  r, i, fold_, and x_idx== 2 58 2 1|5;  r, i, fold_, and x_idx== 2 58 3 1|5;  r, i, fold_, and x_idx== 2 58 4 1|5;  r, i, fold_, and x_idx== 2 59 0 7|9;  train_y.shape, train_y/test_y.mean: (52, 9) 0.13 0.28
r, i, fold_, and x_idx== 2 59 1 7|9;  r, i, fold_, and x_idx== 2 59 2 7|9;  r, i, fold_, and x_idx== 2 59 3 7|9;  r, i, fold_, and x_idx== 2 59 4 7|9;  r, i, fold_, and x_idx== 2 60 0 6|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.12 0.3
r, i, fold_, and

r, i, fold_, and x_idx== 2 90 1 7|9;  r, i, fold_, and x_idx== 2 90 2 7|9;  r, i, fold_, and x_idx== 2 90 3 7|9;  r, i, fold_, and x_idx== 2 90 4 7|9;  r, i, fold_, and x_idx== 2 91 0 6|7;  train_y.shape, train_y/test_y.mean: (52, 9) 0.08 0.25
r, i, fold_, and x_idx== 2 91 1 6|7;  r, i, fold_, and x_idx== 2 91 2 6|7;  r, i, fold_, and x_idx== 2 91 3 6|7;  r, i, fold_, and x_idx== 2 91 4 6|7;  r, i, fold_, and x_idx== 2 92 0 6|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.12 0.3
r, i, fold_, and x_idx== 2 92 1 6|8;  r, i, fold_, and x_idx== 2 92 2 6|8;  r, i, fold_, and x_idx== 2 92 3 6|8;  r, i, fold_, and x_idx== 2 92 4 6|8;  r, i, fold_, and x_idx== 2 93 0 1|4;  train_y.shape, train_y/test_y.mean: (52, 9) 0.05 0.26
r, i, fold_, and x_idx== 2 93 1 1|4;  r, i, fold_, and x_idx== 2 93 2 1|4;  r, i, fold_, and x_idx== 2 93 3 1|4;  r, i, fold_, and x_idx== 2 93 4 1|4;  r, i, fold_, and x_idx== 2 94 0 6|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.12 0.3
r, i, fold_, and x_idx== 2

r, i, fold_, and x_idx== 2 123 1 5|9;  r, i, fold_, and x_idx== 2 123 2 5|9;  r, i, fold_, and x_idx== 2 123 3 5|9;  r, i, fold_, and x_idx== 2 123 4 5|9;  r, i, fold_, and x_idx== 2 124 0 6|8;  train_y.shape, train_y/test_y.mean: (52, 9) 0.12 0.3
r, i, fold_, and x_idx== 2 124 1 6|8;  r, i, fold_, and x_idx== 2 124 2 6|8;  r, i, fold_, and x_idx== 2 124 3 6|8;  r, i, fold_, and x_idx== 2 124 4 6|8;  r, i, fold_, and x_idx== 2 125 0 3|9;  train_y.shape, train_y/test_y.mean: (52, 9) 0.09 0.3
r, i, fold_, and x_idx== 2 125 1 3|9;  r, i, fold_, and x_idx== 2 125 2 3|9;  r, i, fold_, and x_idx== 2 125 3 3|9;  r, i, fold_, and x_idx== 2 125 4 3|9;  r, i, fold_, and x_idx== 2 126 0 4|5;  train_y.shape, train_y/test_y.mean: (52, 9) 0.09 0.23
r, i, fold_, and x_idx== 2 126 1 4|5;  r, i, fold_, and x_idx== 2 126 2 4|5;  r, i, fold_, and x_idx== 2 126 3 4|5;  r, i, fold_, and x_idx== 2 126 4 4|5;  r, i, fold_, and x_idx== 2 127 0 0|6;  train_y.shape, train_y/test_y.mean: (52, 9) -0.03 0.16
r, i,

...

/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:673: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n

r, i, fold_, and x_idx== 10 254 1 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 254 2 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 254 3 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 254 4 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 255 0 0|1|2|3|4|5|6|7|9|10;  train_y.shape, train_y/test_y.mean: (52, 1) -0.35 -0.38
r, i, fold_, and x_idx== 10 255 1 0|1|2|3|4|5|6|7|9|10;  r, i, fold_, and x_idx== 10 255 2 0|1|2|3|4|5|6|7|9|10;  r, i, fold_, and x_idx== 10 255 3 0|1|2|3|4|5|6|7|9|10;  r, i, fold_, and x_idx== 10 255 4 0|1|2|3|4|5|6|7|9|10;  r, i, fold_, and x_idx== 10 256 0 1|2|3|4|5|6|7|8|9|10;  train_y.shape, train_y/test_y.mean: (52, 1) 0.92 1.0
r, i, fold_, and x_idx== 10 256 1 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 256 2 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 256 3 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 256 4 1|2|3|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 257 0 0|1|2|3|4|5|6|7|9|10;  train_y.shape, train_y/test_y.mean: 

r, i, fold_, and x_idx== 10 282 1 0|1|2|3|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 282 2 0|1|2|3|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 282 3 0|1|2|3|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 282 4 0|1|2|3|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 283 0 0|1|2|3|4|5|6|7|8|9;  train_y.shape, train_y/test_y.mean: (52, 1) -0.54 -0.54
r, i, fold_, and x_idx== 10 283 1 0|1|2|3|4|5|6|7|8|9;  r, i, fold_, and x_idx== 10 283 2 0|1|2|3|4|5|6|7|8|9;  r, i, fold_, and x_idx== 10 283 3 0|1|2|3|4|5|6|7|8|9;  r, i, fold_, and x_idx== 10 283 4 0|1|2|3|4|5|6|7|8|9;  r, i, fold_, and x_idx== 10 284 0 0|1|2|3|4|5|6|7|8|10;  train_y.shape, train_y/test_y.mean: (52, 1) -0.12 -0.23
r, i, fold_, and x_idx== 10 284 1 0|1|2|3|4|5|6|7|8|10;  r, i, fold_, and x_idx== 10 284 2 0|1|2|3|4|5|6|7|8|10;  r, i, fold_, and x_idx== 10 284 3 0|1|2|3|4|5|6|7|8|10;  r, i, fold_, and x_idx== 10 284 4 0|1|2|3|4|5|6|7|8|10;  r, i, fold_, and x_idx== 10 285 0 0|1|2|3|4|5|6|7|8|10;  train_y.shape, train_y/test_y.mean: (5

/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:673: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/konrad/mambaforge/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n

r, i, fold_, and x_idx== 10 296 1 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 296 2 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 296 3 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 296 4 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 297 0 0|1|2|4|5|6|7|8|9|10;  train_y.shape, train_y/test_y.mean: (52, 1) 0.27 0.38
r, i, fold_, and x_idx== 10 297 1 0|1|2|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 297 2 0|1|2|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 297 3 0|1|2|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 297 4 0|1|2|4|5|6|7|8|9|10;  r, i, fold_, and x_idx== 10 298 0 0|1|2|3|4|5|6|8|9|10;  train_y.shape, train_y/test_y.mean: (52, 1) 0.0 -0.08
r, i, fold_, and x_idx== 10 298 1 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 298 2 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 298 3 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 298 4 0|1|2|3|4|5|6|8|9|10;  r, i, fold_, and x_idx== 10 299 0 0|1|2|3|4|5|6|8|9|10;  train_y.shape, train_y/test_y.mean: (

In [18]:
pd.DataFrame.from_records(stats_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'tn|fp|fn|tp']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-stats.csv', index=False)
pd.DataFrame.from_records(acc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'accuracy']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-acc.csv', index=False)
pd.DataFrame.from_records(b_acc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'balanced_accuracy']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-bacc.csv', index=False)
pd.DataFrame.from_records(f1_weight_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'f1_score_weighted']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-f1weighted.csv', index=False)
pd.DataFrame.from_records(jaccard_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'jaccard_score']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-jaccard.csv', index=False)
pd.DataFrame.from_records(cohen_kappa_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'cohen_kappa_score']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-cohen_kappa.csv', index=False)
pd.DataFrame.from_records(mcc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'mcc']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-mcc.csv', index=False)
pd.DataFrame.from_records(mcc_js,
                          columns=['# of predictors', 'i', 'predictors', 'response', 'fold', 'mcc']).to_csv(
    f'../Results/MIP_marine_baseline_20231213_stats-mcc_js.csv', index=False)